In [1]:
# Install widgets package if missing.
#! pip install ipywidgets

In [2]:
# Import sliders, which will be used in interactive charts.
from ipywidgets import interact, fixed
from ipywidgets.widgets import FloatRangeSlider, IntRangeSlider, FloatSlider

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import binomial_options_pricing_model.bopm as bopm

In [4]:
# Slider for each parameter of the BOPM model.
# Here you can change min/max of the parameter. Also step of the slider.
r_slider = FloatSlider(value=.02, step=0.01, min=0, max=0.25, description='Risk-free rate:')
S_slider = FloatSlider(value=50, step=0.5, min=40, max=60, description='Spot price:')
K_slider = FloatSlider(value=48, step=0.5, min=40, max=60, description='Strike price:')
delta_t_slider = FloatSlider(value=1/12, step=0.05, min=1/365, max=1, description='Time step (in years):')
T_slider = FloatSlider(value=2, min=0.5, max=10, description='Maturity (in years):')
sigma_slider = FloatSlider(value=.3, step=0.05, min=0, max=1, description='Volatility:')

# Slider for the range of the domain (in plots) of parameters. 
# It's an interval [domain[0]*param, domain[1]*param].
domain_range_slider = FloatRangeSlider(value=(0.8, 1.2), step=0.1, min=0.1, max=3, description='Domain:')

# One-dimensional slices
Price of an option is a multivariable function:
$ V \left( r, S_0, K, \Delta t, T, \sigma \right) $.

First step in analyzing price of an option is to look at one-dimensional slices of price function. 

Below I'm constructing functions used later for plotting: $V_r(r)$, $V_{S_0}(S_0)$, $V_K(K)$, $V_{\Delta t}(\Delta t)$, $V_T(T)$, $V_{\sigma}(\sigma)$.

In [5]:
# One-dimensional slices of the price function.

# They are vectorized (a loop), for easy plotting later.
# Default values of constant parameters are taken from the sliders.
# This way, each plot will change after change in one parameter.

from binomial_options_pricing_model.bopm_crr_price_functions import vectorize

def v_r(r: np.ndarray, american = False, call = True) -> np.ndarray:
    func = lambda arg: bopm.crr_price_option(arg, S_slider.value, K_slider.value,
                                             delta_t_slider.value,T_slider.value,
                                             sigma_slider.value, american, call)[0]
    return vectorize(func, r)

def v_s(S: np.ndarray, american = False, call = True) -> np.ndarray:
    func = lambda arg: bopm.crr_price_option(r_slider.value, arg, K_slider.value,
                                             delta_t_slider.value,T_slider.value,
                                             sigma_slider.value, american, call)[0]
    return vectorize(func, S)

def v_k(K: np.ndarray, american = False, call = True) -> np.ndarray:
    func = lambda arg: bopm.crr_price_option(r_slider.value, S_slider.value, arg,
                                             delta_t_slider.value,T_slider.value,
                                             sigma_slider.value, american, call)[0]
    return vectorize(func, K)

def v_delta_t(delta_t: np.ndarray, american = False, call = True) -> np.ndarray:
    func = lambda arg: bopm.crr_price_option(r_slider.value, S_slider.value, K_slider.value,
                                               arg, T_slider.value,
                                               sigma_slider.value, american, call)[0]
    return vectorize(func, delta_t)
    
def v_t(T: np.ndarray, american = False, call = True) -> np.ndarray:
    func = lambda arg: bopm.crr_price_option(r_slider.value, S_slider.value, K_slider.value,
                                             delta_t_slider.value, arg,
                                             sigma_slider.value, american, call)[0]
    return vectorize(func, T)
    
def v_sigma(sigma: np.ndarray, american = False, call = True) -> np.ndarray:
    func = lambda arg: bopm.crr_price_option(r_slider.value, S_slider.value, K_slider.value,
                                             delta_t_slider.value,T_slider.value,
                                             arg, american, call)[0]
    return vectorize(func, sigma)

price_functions = v_r, v_s, v_k, v_delta_t, v_t, v_sigma

In [6]:
from binomial_options_pricing_model.plot_crr import plot_crr_price_functions

interact(plot_crr_price_functions, r=r_slider, S=S_slider, K=K_slider, delta_t=delta_t_slider,
         T=T_slider, sigma=sigma_slider, domain=domain_range_slider,
         price_functions=fixed(price_functions))
plt.rcParams['figure.figsize'] = [15, 10]

interactive(children=(FloatSlider(value=0.02, description='Risk-free rate:', max=0.25, step=0.01), FloatSlider…

# Analysis
First conclusion, american put has always higher price.

Second conclusion, price of an american and european call are always the same. Quick proof.

Price function of time parameters is bumpy. Is it because there is integer number of steps in the tree?

Test out limit of domain, like r=0

what if K increases/decreases